In [1]:
%pip install -q transformers datasets evaluate accelerate scikit-learn torch


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Imports

In [1]:
from datasets import (
    load_dataset, 
    DatasetDict, 
)
import torch
from typing import Dict, Any
from transformers import (
    AutoModelForMaskedLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    TrainingArguments, 
    Trainer,
    pipeline
)

2023-07-06 18:28:11.695599: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-06 18:28:11.737886: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-06 18:28:11.739189: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 18:28:12.414187: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model_checkpoint = "distilroberta-base"

## Load Dataset

In [3]:
# Task 1 - Load the train and test splits from ag_news. Randomly select 10% of the training set as validation.

SEED = 42

dataset = load_dataset("ag_news")
dataset = dataset.shuffle(SEED)

# dataset["train"] = load_dataset("ag_news", split="train[:4000]") # Note: This is useful for sanity checking the training process. Comment out/Uncomment as necessary

train_val_dataset = dataset["train"].train_test_split(test_size=0.1, seed=SEED)  # Split training set into training and validation set

# Construct new dataset object from old test, new train and new validation sets
dataset = DatasetDict({
    'train': train_val_dataset["train"],
    'test': dataset["test"],
    'val': train_val_dataset['test']
})

Found cached dataset ag_news (/home/nano/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/nano/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-dd0ff9596fea92b0.arrow
Loading cached shuffled indices for dataset at /home/nano/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-12f3c4e4bf422cce.arrow
Loading cached split indices for dataset at /home/nano/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-8f4a15d7afc8e025.arrow and /home/nano/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-c359d8246d1cf46f.arrow


## Preprocessing Function

In [23]:
# Task 3
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess_function(example, seq_len: int):
    result = tokenizer(
        example["text"], 
        truncation=True, 
        padding="max_length", 
        max_length=seq_len,
        return_special_tokens_mask=True,
    )
    result['label'] = result['input_ids'].copy() # Copy input_ids for later use during training
    return result

encoded_ds = dataset.map(
    preprocess_function, 
    batched=True,
    fn_kwargs={"seq_len": 256},
    remove_columns=['text']
)

Map:   0%|          | 0/108000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

## Data Collator

In [14]:
# Task 4
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
                                                mlm_probability=0.10)

## Load Model

In [19]:
# Task 5
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [21]:
# Task 6
model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

## Define TrainingArguments

In [ ]:
# Task 7
# TODO: Learning Rate Scheduler, Weight Decay

# ['linear', 'cosine', 'cosine_with_restarts', 'polynomial', 'constant', 'constant_with_warmup', 'inverse_sqrt', 'reduce_lr_on_plateau']
lr_scheduler_type = "linear"

training_args = TrainingArguments(output_dir = './checkpoints/',
                                  do_train=True,
                                  do_eval=True,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32,
                                  learning_rate=2e-5,
                                  evaluation_strategy="epoch",
                                  num_train_epochs=5,
                                  load_best_model_at_end=True,
                                  save_strategy="epoch",
                                  lr_scheduler_type=lr_scheduler_type,
                                  weight_decay=0.1,
                                  load_best_model_at_end=True,
                                  )

## Define Trainer

In [ ]:
# Task 8
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset = encoded_ds["train"],
                  eval_dataset = encoded_ds["test"],
                  data_collator=data_collator)

## Train Model

In [ ]:
# Task 9

# TODO: Hyper parameter tuning
#   - batch size
#   - number of epochs
#   - weight decay
#   - learning rate
# Note: Should be executed in Google Colab
# Note: Does not yet work as intended... Training loss does not seem to go down...

trainer.train()

## Evaluation on Validation and Test Splits with Perplexity

In [ ]:
# Task 10
# TODO: Calculate perplexity on validation and test splits
# Note: Check out this: https://huggingface.co/docs/transformers/perplexity

# from evaluate import load
# perplexity = load("perplexity", module_type="metric")

# predictions_train = trainer(encoded_ds["test"])
# predictions_val = trainer(encoded_ds["val"])

# results_train = perplexity.compute(predictions=predictions_train)
# results_val = perplexity.compute(predictions=predictions_val)

# print(results_train)
# print(results_val)

## Inference

In [ ]:
# Task 11

text = "E-mail scam targets police chief Wiltshire Police warns about <mask> after its fraud squad chief was targeted."

mask_filler = pipeline('mask-filler', trainer)
mask_filler(text, top_k=5)